<a href="https://colab.research.google.com/github/edgaracabral/PoDHackaton02_2023/blob/main/PoD%20FW%20Hackaton_02%2004%20Escoragem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Código de Escoragem Batch

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
# Bibliotecas necessárias para o projeto
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
import pickle
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, roc_auc_score

# Redirecione para o folder on pod_academy_functions.py is localizado
%cd /content/drive/MyDrive/PoD Academy/modelos/Hackaton_DS_2023/PoD Framework
import pod_academy_functions as pod

/content/drive/MyDrive/PoD Academy/modelos/Hackaton_DS_2023/PoD Framework


In [98]:
# Leitura dos dados a serem escorados
%cd /content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023

df_train_00 = pd.read_csv('application_train.csv')
print(f'df_train_00.shape = {df_train_00.shape}')

df_test_00 = pd.read_csv('application_test.csv')
print(f'df_test_00.shape = {df_test_00.shape}')

/content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023
df_train_00.shape = (215257, 172)
df_test_00.shape = (92254, 171)


In [99]:
# Renomeie as colunas de id e target para o padrão do Framework (id, target)
df_train_00.rename(columns={'SK_ID_CURR': 'id', 'TARGET': 'target'}, inplace=True)
df_test_00.rename(columns={'SK_ID_CURR': 'id'}, inplace=True)

In [100]:
# Criando um novo dataframe baseado no original
df_train_01 = df_train_00.copy()
df_test_01 = df_test_00.copy()

In [101]:
# A partir de agora, todos os artefatos gerados serão colocados dentro da pasta dataprep
%cd /content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023/dataprep

/content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023/dataprep


In [102]:
# Carregar lista de variáveis que foram excluidas por execesso de nulos
with open('prd_drop_nullvars_hktn.pkl', 'rb') as f:
    loaded_drop_null_feat = pickle.load(f)
loaded_drop_null_feat

[]

In [103]:
# Retirando as variáveis excluídas por nulos
df_train_02 = df_train_01.drop(axis=1,columns=loaded_drop_null_feat)
df_test_02 = df_test_01.drop(axis=1,columns=loaded_drop_null_feat)

In [104]:
df_test_02.head()

,id,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,384575,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,...,0.633686,0.664401,0.087885,0.350320,0.247838,0.700314,0.709003,0.625105,0.847218,0.445958
1,214010,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,...,0.249826,0.569889,0.161183,0.932276,0.720287,0.251879,0.439528,0.269992,0.547486,0.657934
2,142232,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,...,0.081305,0.025186,0.391566,0.737325,0.090995,0.327031,0.787739,0.820345,0.523118,0.339553
3,389171,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,...,0.062781,0.129004,0.054182,0.084110,0.619411,0.115114,0.314785,0.659989,0.864239,0.315243
4,283617,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,234000.0,...,0.730765,0.292157,0.155718,0.276252,0.666383,0.140511,0.598934,0.631357,0.412754,0.628068


In [105]:
# Retirar ID e Target das tabelas (para escoragem não é necessário e em produção não teremos target)
df_train_02 = df_train_02.drop(axis=1, columns=['id','target'])
df_test_02 = df_test_02.drop(axis=1, columns=['id'])

df_test_02.shape,df_train_02.shape

((92254, 170), (215257, 170))

In [106]:
df_train_02.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,Cash loans,F,N,N,0,157500.0,706410.0,67072.5,679500.0,Unaccompanied,...,0.824762,0.333516,0.293260,0.564878,0.115058,0.655605,0.415562,0.092643,0.723331,0.796523
1,Cash loans,F,Y,Y,1,121500.0,545040.0,25407.0,450000.0,Unaccompanied,...,0.416260,0.404293,0.137944,0.457971,0.303691,0.215059,0.838892,0.608335,0.585643,0.298456
2,Cash loans,M,Y,Y,1,225000.0,942300.0,27679.5,675000.0,Unaccompanied,...,0.037711,0.124465,0.091840,0.364601,0.978220,0.520309,0.594523,0.559650,0.361873,0.254804
3,Revolving loans,F,N,Y,2,144000.0,180000.0,9000.0,180000.0,Unaccompanied,...,0.784630,0.831403,0.210872,0.049639,0.814219,0.830179,0.755163,0.216664,0.603002,0.429001
4,Cash loans,F,N,Y,0,112500.0,729792.0,37390.5,630000.0,Unaccompanied,...,0.265381,0.655344,0.668705,0.171391,0.335702,0.585494,0.619551,0.686738,0.540449,0.343632


In [107]:
# Carregar as medias das variaveis da base de treino
with open('prd_fillna_hktn.pkl', 'rb') as f:
  loaded_means = pickle.load(f)

# Aplicar imputação de nulos na base de treino
df_train_02 = pod.pod_custom_fillna_prod(df_train_02,loaded_means)

# Aplicar imputação de nulos na base de teste
df_test_02 = pod.pod_custom_fillna_prod(df_test_02,loaded_means)

df_test_02.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,Unaccompanied,...,0.633686,0.664401,0.087885,0.350320,0.247838,0.700314,0.709003,0.625105,0.847218,0.445958
1,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,Unaccompanied,...,0.249826,0.569889,0.161183,0.932276,0.720287,0.251879,0.439528,0.269992,0.547486,0.657934
2,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,Unaccompanied,...,0.081305,0.025186,0.391566,0.737325,0.090995,0.327031,0.787739,0.820345,0.523118,0.339553
3,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,Unaccompanied,...,0.062781,0.129004,0.054182,0.084110,0.619411,0.115114,0.314785,0.659989,0.864239,0.315243
4,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,234000.0,Unaccompanied,...,0.730765,0.292157,0.155718,0.276252,0.666383,0.140511,0.598934,0.631357,0.412754,0.628068


In [108]:
# Carregar os encoders e a lista de colunas
with open('prd_labelenc_hktn.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

# Aplicando para base de treino
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        df_train_02[col] = loaded_encoders[col].transform(df_train_02[col])

# Aplicando para base de teste
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        df_test_02[col] = loaded_encoders[col].transform(df_test_02[col])

In [109]:
# Carregar o encoder e a lista de colunas
with open('prd_onehotenc_hktn.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoder = loaded_data['encoder']
loaded_columns = loaded_data['columns']

# Aplicando para base completa de treino
encoded_data_test = loaded_encoder.transform(df_train_02[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=df_train_02.index)
df_train_03 = pd.concat([df_train_02.drop(loaded_columns, axis=1), encoded_df_test], axis=1)

# Aplicando para base de teste
encoded_data_test = loaded_encoder.transform(df_test_02[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=df_test_02.index)
df_test_03 = pd.concat([df_test_02.drop(loaded_columns, axis=1), encoded_df_test], axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [110]:
df_train_03.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,FLAG_OWN_REALTY_Y,FLAG_OWN_CAR_Y,CODE_GENDER_M,CODE_GENDER_XNA,NAME_CONTRACT_TYPE_Revolving loans
0,0,157500.0,706410.0,67072.5,679500.0,0.032561,-14653,-2062.0,-8599.0,-2087.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,121500.0,545040.0,25407.0,450000.0,0.007114,-13995,-2246.0,-348.0,-172.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2,1,225000.0,942300.0,27679.5,675000.0,0.022625,-21687,-1335.0,-6306.0,-4026.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
3,2,144000.0,180000.0,9000.0,180000.0,0.006629,-13071,-2292.0,-742.0,-1201.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,112500.0,729792.0,37390.5,630000.0,0.046220,-19666,365243.0,-169.0,-3112.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [111]:
df_train_04 = df_train_03
df_test_04 = df_test_03

In [112]:
# Carregar o scaler
with open('prd_scaler_hktn.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# Aplicando na base de treino
df_train_04_scaled = loaded_scaler.transform(df_train_04)
df_train_05 = pd.DataFrame(df_train_04_scaled, columns=df_train_04.columns, index=df_train_04.index)

# Aplicando na base de teste
df_test_04s = loaded_scaler.transform(df_test_04)
df_test_05 = pd.DataFrame(df_test_04s, columns=df_test_04.columns, index=df_test_04.index)
df_test_05.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,FLAG_OWN_REALTY_Y,FLAG_OWN_CAR_Y,CODE_GENDER_M,CODE_GENDER_XNA,NAME_CONTRACT_TYPE_Revolving loans
0,2.198670,0.353377,-0.333618,1.752998,-0.326438,-0.812271,0.628759,-0.456159,1.231927,-0.871924,...,-0.467613,-0.275573,-0.007287,-0.009289,-1.032565,-1.508732,1.387366,1.384894,-0.002576,-0.3251
1,-0.577795,0.727042,1.692735,1.498934,1.731686,-1.013500,0.288773,-0.458846,0.955678,-1.030381,...,-0.467613,-0.275573,-0.007287,-0.009289,-1.032565,0.662808,1.387366,-0.722077,-0.002576,-0.3251
2,-0.577795,0.311858,-0.260279,0.822460,-0.119408,1.082825,-0.429536,-0.455288,0.701006,1.014976,...,-0.467613,-0.275573,-0.007287,-0.009289,0.968462,-1.508732,1.387366,-0.722077,-0.002576,-0.3251
3,-0.577795,0.727042,-0.856824,-0.152021,-0.850103,1.838198,-0.824158,-0.500244,-1.757700,-0.110136,...,-0.467613,3.628804,-0.007287,-0.009289,-1.032565,0.662808,-0.720790,-0.722077,-0.002576,-0.3251
4,-0.577795,-0.518510,-0.724176,-0.776196,-0.825747,-0.144402,-0.985083,-0.458590,-0.659516,1.656097,...,-0.467613,-0.275573,-0.007287,-0.009289,0.968462,0.662808,-0.720790,1.384894,-0.002576,-0.3251


In [113]:
# Carregar lista de variáveis que passaram pelo Feature Selection (utilizadas no treinamento do modelo)
with open('prd_list_features_hktn.pkl', 'rb') as f:
    loaded_features = pickle.load(f)

loaded_features

['DAYS_BIRTH',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'var_9',
 'var_19',
 'var_20',
 'var_23',
 'var_25',
 'var_27',
 'var_32',
 'var_33']

In [114]:
abt_train = df_train_05[loaded_features]
abt_test = df_test_05[loaded_features]

In [115]:
abt_train.head()

,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,var_9,var_19,var_20,var_23,var_25,var_27,var_32,var_33
0,0.317469,-1.028606,0.599937,7.968324e-16,0.618225,-1.666068,0.063861,-0.230093,0.736679,0.403026,-0.117495,-1.637559,0.614048,0.060833
1,0.468523,1.313979,1.869584,6.478897e-01,0.951529,0.700575,1.470035,1.051540,-1.206514,-0.849304,-0.525654,0.846057,0.163177,1.113220
2,-1.297292,-0.377588,-0.685621,7.968324e-16,0.803097,0.554575,0.728328,0.782789,-1.192338,0.241685,-1.715574,-1.045641,-0.351992,1.382372
3,0.680641,1.202116,1.187356,7.968324e-16,-1.047897,-0.476475,-0.535640,-1.632014,1.000297,-0.874321,0.642610,1.053783,-0.468563,-0.761145
4,-0.833341,1.364799,-0.079638,6.918355e-01,-0.044888,-1.556420,-0.179153,-0.177546,-0.704765,-0.812556,0.538060,-1.267725,-0.098190,-1.574394


In [116]:
# Vá para o folder de modelos
%cd /content/drive/MyDrive/PoD Academy/modelos/Hackaton_DS_2023/PoD Framework/modelos

# Carregando modelo campeão
with open('best_model_rf.pkl', 'rb') as f:
  loaded_model = pickle.load(f)

loaded_model

/content/drive/MyDrive/PoD Academy/modelos/Hackaton_DS_2023/PoD Framework/modelos


RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=0.01)

In [117]:
# Escorando base de treino

predprob = loaded_model.predict_proba(abt_train)
predict = loaded_model.predict(abt_train)
abt_train['score_1']=predprob[:,1]
abt_train['class']=predict

# Escorando base de teste

predprob = loaded_model.predict_proba(abt_test)
predict = loaded_model.predict(abt_test)
abt_test['score_1']=predprob[:,1]
abt_test['class']=predict

<ipython-input-117-83503806e553>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_train['score_1']=predprob[:,1]
<ipython-input-117-83503806e553>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_train['class']=predict
<ipython-input-117-83503806e553>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [118]:
abt_train = abt_train.merge(df_train_00[['id','target']], left_index=True, right_index=True, how='inner')
abt_test = abt_test.merge(df_test_00[['id']], left_index=True, right_index=True, how='inner')

In [119]:
abt_train.head()

,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,var_9,var_19,var_20,var_23,var_25,var_27,var_32,var_33,score_1,class,id,target
0,0.317469,-1.028606,0.599937,7.968324e-16,0.618225,-1.666068,0.063861,-0.230093,0.736679,0.403026,-0.117495,-1.637559,0.614048,0.060833,0.568700,1,247330,0
1,0.468523,1.313979,1.869584,6.478897e-01,0.951529,0.700575,1.470035,1.051540,-1.206514,-0.849304,-0.525654,0.846057,0.163177,1.113220,0.361165,0,425716,1
2,-1.297292,-0.377588,-0.685621,7.968324e-16,0.803097,0.554575,0.728328,0.782789,-1.192338,0.241685,-1.715574,-1.045641,-0.351992,1.382372,0.318757,0,331625,0
3,0.680641,1.202116,1.187356,7.968324e-16,-1.047897,-0.476475,-0.535640,-1.632014,1.000297,-0.874321,0.642610,1.053783,-0.468563,-0.761145,0.649034,1,455397,0
4,-0.833341,1.364799,-0.079638,6.918355e-01,-0.044888,-1.556420,-0.179153,-0.177546,-0.704765,-0.812556,0.538060,-1.267725,-0.098190,-1.574394,0.533738,1,449114,0


In [120]:
abt_test.head()

,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,var_9,var_19,var_20,var_23,var_25,var_27,var_32,var_33,score_1,class,id
0,0.628759,1.231927,-0.871924,1.239450e+00,0.473892,-2.924463,-1.178207,-1.401094,-1.151752,0.946348,0.955825,-1.523299,1.544985,0.622043,0.474250,0,384575
1,0.288773,0.955678,-1.030381,-5.193365e-01,-0.467427,1.154051,0.192706,-0.554271,0.791193,-0.327802,0.968410,1.184114,-0.118517,0.018045,0.457019,0,214010
2,-0.429536,0.701006,1.014976,1.726345e-01,0.090461,-1.735591,1.002028,-1.086760,1.023441,1.162285,-0.456197,-0.652544,-0.952576,0.476660,0.556688,1,142232
3,-0.824158,-1.757700,-0.110136,7.968324e-16,0.938545,0.593875,0.148207,-0.840843,1.007324,1.054237,-1.337287,0.293598,1.385024,0.112349,0.309457,0,389171
4,-0.985083,-0.659516,1.656097,1.088753e+00,0.242135,0.719743,-1.309624,-1.594943,-1.053740,1.562200,1.181377,-1.672768,1.220907,0.058500,0.308837,0,283617


In [121]:
#### Preparando para envio para o Kaggle
abt_envio_kaggle = abt_test[['id','score_1']]
abt_envio_kaggle.rename(columns={'score_1':'target'},inplace=True)
abt_envio_kaggle.head()

<ipython-input-121-652e3fc3c50b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_envio_kaggle.rename(columns={'score_1':'target'},inplace=True)


,id,target
0,384575,0.474250
1,214010,0.457019
2,142232,0.556688
3,389171,0.309457
4,283617,0.308837


In [122]:
# Renomeie as colunas de id e target para o formato sample_submission.csv
abt_envio_kaggle.rename(columns={'id': 'ID', 'target': 'TARGET'}, inplace=True)
abt_envio_kaggle.head()

<ipython-input-122-a37fdaa4bffe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_envio_kaggle.rename(columns={'id': 'ID', 'target': 'TARGET'}, inplace=True)


,ID,TARGET
0,384575,0.474250
1,214010,0.457019
2,142232,0.556688
3,389171,0.309457
4,283617,0.308837


In [123]:
# arquivo final será salvado no dados/../dataprep
%cd /content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023/dataprep

#### Salvando como arquivo csv
abt_envio_kaggle.to_csv('abt_scored_002_rf.csv', index=False)

/content/drive/MyDrive/PoD Academy/dados/Kaggle/Hackaton_DS_2023/dataprep
